In [10]:
import geopandas as gpd
import pandas as pd 
from shapely.ops import unary_union
import osmnx as ox
import os 
import warnings 
warnings.filterwarnings("ignore", category=DeprecationWarning)

CITIES = ['Lyon','Villeurbanne','Décines-Charpieu','Vaulx-En-Velin']
FOLDER_PATH = '../../../../data/rrochas/prediction_validation' 

def split_multi_polygon(gdf):
    '''
    From a gdf which contains some Multi-Polygons, split them as a Series of Polygon:
    '''
    gdf_multi_poly = gdf[gdf.geometry.type == 'MultiPolygon']
    gdf_poly = gdf[gdf.geometry.type != 'MultiPolygon']

    for idx,row in gdf_multi_poly.iterrows():
        list_of_poly = list(row.geometry.geoms)
        tmp_list = []
        for poly in list_of_poly :
            dict_row = {key:value for key,value in row.items()}
            dict_row.update({'geometry' : poly,'element_type':row.name[0],'osmid':row.name[1]})
            tmp_list.append(dict_row)
        splitted_multi_poly = gpd.GeoDataFrame(tmp_list)
        splitted_multi_poly = splitted_multi_poly.set_index(['element_type','osmid'])

        #print('list_of_poly: ',len(list_of_poly))
        #print(list_of_poly)
        #print('splitted_multi_poly: ',splitted_multi_poly.shape)
        #print(splitted_multi_poly) 
        #splitted_multi_poly['geometry'] = list_of_poly
        
        # Concat to the init gdf:
        gdf_poly = gpd.GeoDataFrame(pd.concat([gdf_poly,splitted_multi_poly]),crs = gdf.crs)

    return(gdf_poly)


def group_adjacent_polygon(gdf):
    '''
    Créer une fonction qui regroupe les polygon adjacent pour en crée un seul. 
    Règle le problème où certaines géoémtrie comme les parcs sont subdivisé, 
    alors qu'on veut la géométrie du parc entier.
    '''
    gdf = split_multi_polygon(gdf)
    gdf = gdf.reset_index()  # set index from 0 to n 
    gdf = gdf.drop(columns=['element_type'])
    # Identifier les polygones en contact
    contact_indices = []
    for i in range(len(gdf)): #pass through index 
        for j in range(i + 1, len(gdf)):
            # Two polygon might intersect 
            try:
                condition = (gdf.geometry.loc[i].intersects(gdf.geometry.loc[j]))  #.values[0] #.values[0]
            # Tackle close polygon or point + polygon ..
            except: 
                condition = (gdf.geometry.loc[i].touches(gdf.geometry.loc[j])) 
            if condition:
                contact_indices.append(set([i,j]))


    # Regrouper l'ensemble des contact indices qui ont un élément en commmun : 
    grouped_geometry = {}
    for nb_iter,(i,j) in enumerate(contact_indices):
        group_exists = False
        id_set_to_remove = []
        new_set = set([i,j]) 
        for k,grouped_geom in grouped_geometry.items():
            # Si i ou j est déjà dans un groupe, ajouter l'autre dans le groupe: 

            # Parfois jusqu'à 2 id à retirer
            if (i in grouped_geom) or (j in grouped_geom):
                new_set = list(set(grouped_geom)|set(new_set))
                id_set_to_remove.append(k)
                group_exists = True
                        
        # Si aucun groupe n'a été trouvé, on en crée un 
        if not group_exists:
            new_indice = len(grouped_geometry)
            grouped_geometry[new_indice] = [i,j]
        else:
            for id_set in id_set_to_remove:
                # remove
                grouped_geometry.pop(id_set)
            # reorganise
            grouped_geometry = {i: grouped_geometry[key] for i, key in enumerate(grouped_geometry)}
            # add new item:
            grouped_geometry[len(grouped_geometry)] = new_set

    # Pour chaque grouped_geometry, les regrouper (gdf_to_add), et supprimer les geométrie non regroupé de gdf:
    gdf_to_add = gpd.GeoDataFrame()
    for k,grouped_geom in grouped_geometry.items():
        '''gdf.loc[grouped_geom].geometry'''
        geometries_to_merge = gdf.geometry[grouped_geom] 
        merged_geom = unary_union(geometries_to_merge)
        gdf_to_add_i = gpd.GeoDataFrame(dict(geometry = [merged_geom]),crs = gdf.crs)
        gdf_to_add_i['name'] = gdf.loc[grouped_geom[0]]['name']
        gdf_to_add_i['osmid'] = gdf.loc[grouped_geom[0]]['osmid']
        gdf_to_add = gpd.GeoDataFrame(pd.concat([gdf_to_add,gdf_to_add_i]),crs = gdf.crs)

        # Supprime les un-grouped geometry :
        gdf = gdf.drop(grouped_geom)

    new_gdf = gpd.GeoDataFrame(pd.concat([gdf,gdf_to_add]), crs=gdf.crs)
    return(new_gdf)

def concat_gdf_through_cities(CITIES,tag_type,sub_tag,sports_tags):
    gdf = gpd.GeoDataFrame()
    for city in CITIES:
        place_name = f"{city}, France"
        tags = {tag_type: sub_tag}
        with warnings.catch_warnings():
            warnings.filterwarnings("ignore", category=DeprecationWarning)
            try : 
                gdf_i = ox.features_from_place(place_name, tags=tags)
            except:
                print(f'No {tag_type}-{sub_tag} within {place_name}')
                gdf_i = None
            if gdf_i is not None:    
                if sports_tags is not None:
                    gdf_i = gdf_i[gdf_i.sport.isin(sports_tags)]
                
                gdf = gpd.GeoDataFrame(pd.concat([gdf,gdf_i]), crs=gdf_i.crs)
    return gdf

def get_POIs_from_cities(CITIES,tag_type,sub_tag,sports_tags = None):
    gdf = concat_gdf_through_cities(CITIES,tag_type,sub_tag,sports_tags)
    gdf = gdf[['name','geometry']]
    gdf = group_adjacent_polygon(gdf)
    gdf['type'] = tag_type
    gdf['sub_type'] = sub_tag
    return(gdf)

# Get POIs 'Leisure'
'Leisure' contient les géométries (polygon/multipolygon) des `Parcs`et des `Stades`

### Génère les POIs parcs : 
Rcupère les POIs de parc, groupe les sous-geometrie qui constitue un parc, et retourne uniquement les grands parcs:

In [11]:
tag_type = 'leisure'
sub_tag = 'park'
gdf_park = get_POIs_from_cities(CITIES,tag_type,sub_tag,sports_tags= None)
gdf_park = gdf_park.to_crs('epsg:2154')
gdf_park = gdf_park[gdf_park.geometry.area > 1e4]
gdf_park = gdf_park.to_crs('epsg:4326')
gdf_park.head()


/root/anaconda3/envs/osmnx/lib/python3.12/site-packages/pandas/core/frame.py:717: DeprecationWarning: Passing a BlockManager to GeoDataFrame is deprecated and will raise in a future version. Use public APIs instead.
  warnings.warn(
/root/anaconda3/envs/osmnx/lib/python3.12/site-packages/pandas/core/frame.py:717: DeprecationWarning: Passing a BlockManager to GeoDataFrame is deprecated and will raise in a future version. Use public APIs instead.
  warnings.warn(
/root/anaconda3/envs/osmnx/lib/python3.12/site-packages/pandas/core/frame.py:717: DeprecationWarning: Passing a BlockManager to GeoDataFrame is deprecated and will raise in a future version. Use public APIs instead.
  warnings.warn(
/root/anaconda3/envs/osmnx/lib/python3.12/site-packages/pandas/core/frame.py:717: DeprecationWarning: Passing a BlockManager to GeoDataFrame is deprecated and will raise in a future version. Use public APIs instead.
  warnings.warn(
/root/anaconda3/envs/osmnx/lib/python3.12/site-packages/geopandas/ar

,osmid,name,geometry,type,sub_type
14,22664120,Parc de la Cerisaie,"POLYGON ((4.81750 45.77479, 4.81757 45.77471, ...",leisure,park
15,22727720,Parc Francis Popy,"POLYGON ((4.82349 45.77753, 4.82349 45.77766, ...",leisure,park
17,23269760,Parc Georges Bazin,"POLYGON ((4.88274 45.75170, 4.88274 45.75172, ...",leisure,park
22,24572931,Parc Chambovet,"POLYGON ((4.89546 45.74693, 4.89554 45.74802, ...",leisure,park
23,29176499,Parc des Hauteurs et jardins du Rosaire,"POLYGON ((4.82290 45.76210, 4.82285 45.76210, ...",leisure,park


### Get POIs stadium : 

In [12]:
tag_type = 'leisure'
sub_tag = 'stadium'
sports_tags = ['rugby', 'football','soccer', 'basketball']
gdf_stadium = get_POIs_from_cities(CITIES,tag_type,sub_tag,sports_tags=sports_tags)
gdf_stadium

/root/anaconda3/envs/osmnx/lib/python3.12/site-packages/pandas/core/frame.py:717: DeprecationWarning: Passing a BlockManager to GeoDataFrame is deprecated and will raise in a future version. Use public APIs instead.
  warnings.warn(
/root/anaconda3/envs/osmnx/lib/python3.12/site-packages/pandas/core/frame.py:717: DeprecationWarning: Passing a BlockManager to GeoDataFrame is deprecated and will raise in a future version. Use public APIs instead.
  warnings.warn(
/root/anaconda3/envs/osmnx/lib/python3.12/site-packages/pandas/core/frame.py:717: DeprecationWarning: Passing a BlockManager to GeoDataFrame is deprecated and will raise in a future version. Use public APIs instead.
  warnings.warn(
/root/anaconda3/envs/osmnx/lib/python3.12/site-packages/pandas/core/frame.py:717: DeprecationWarning: Passing a BlockManager to GeoDataFrame is deprecated and will raise in a future version. Use public APIs instead.
  warnings.warn(
/root/anaconda3/envs/osmnx/lib/python3.12/site-packages/pandas/core/

No leisure-stadium within Vaulx-En-Velin, France


,osmid,name,geometry,type,sub_type
0,4585967,Matmut Stadium Gerland,"POLYGON ((4.83278 45.72508, 4.83287 45.72507, ...",leisure,stadium
1,85200814,Astroballe,"POLYGON ((4.90681 45.76680, 4.90699 45.76660, ...",leisure,stadium
2,353267337,Groupama Stadium,"POLYGON ((4.98181 45.76665, 4.98209 45.76665, ...",leisure,stadium


### Génère les POIs Centre commerciaux : 

In [13]:
tag_type = 'shop'

# Tackle mall
sub_tag = 'mall' 
gdf_mall = get_POIs_from_cities(CITIES,tag_type,sub_tag,sports_tags=None)

# Tackle supermarket
sub_tag = 'supermarket' 
gdf_supermarket = get_POIs_from_cities(CITIES,tag_type,sub_tag,sports_tags=None)
gdf_shop = gpd.GeoDataFrame(pd.concat([gdf_mall,gdf_supermarket]),crs = gdf_mall.crs)

# Keep only polygon : 
gdf_shop = gdf_shop[gdf_shop.geometry.geom_type != 'Point']

# Keep only polygon > 2000m2
gdf_shop = gdf_shop.to_crs('epsg:2154')
gdf_shop = gdf_shop[gdf_shop.geometry.area > 2000]
gdf_shop = gdf_shop.to_crs('epsg:4326')
gdf_shop.head()


/root/anaconda3/envs/osmnx/lib/python3.12/site-packages/pandas/core/frame.py:717: DeprecationWarning: Passing a BlockManager to GeoDataFrame is deprecated and will raise in a future version. Use public APIs instead.
  warnings.warn(
/root/anaconda3/envs/osmnx/lib/python3.12/site-packages/pandas/core/frame.py:717: DeprecationWarning: Passing a BlockManager to GeoDataFrame is deprecated and will raise in a future version. Use public APIs instead.
  warnings.warn(
/root/anaconda3/envs/osmnx/lib/python3.12/site-packages/pandas/core/frame.py:717: DeprecationWarning: Passing a BlockManager to GeoDataFrame is deprecated and will raise in a future version. Use public APIs instead.
  warnings.warn(
/root/anaconda3/envs/osmnx/lib/python3.12/site-packages/pandas/core/frame.py:717: DeprecationWarning: Passing a BlockManager to GeoDataFrame is deprecated and will raise in a future version. Use public APIs instead.
  warnings.warn(
/root/anaconda3/envs/osmnx/lib/python3.12/site-packages/pandas/core/

,osmid,name,geometry,type,sub_type
3,440270633,Pôle de Commerces et de Loisirs Confluence,"POLYGON ((4.81908 45.74095, 4.81897 45.74082, ...",shop,mall
4,624026627,Centre commercial Champvert,"POLYGON ((4.79811 45.76262, 4.79811 45.76317, ...",shop,mall
5,300128,Grand Hôtel-Dieu,"POLYGON ((4.83567 45.75806, 4.83583 45.75803, ...",shop,mall
7,395804418,Centre commercial Balzac,"POLYGON ((4.98115 45.77212, 4.98116 45.77149, ...",shop,mall
13,116760770,NaN,"POLYGON ((4.92659 45.76489, 4.92657 45.76509, ...",shop,mall


### Génère les POIs Ecoles et Université : 

In [14]:
tag_type = 'amenity'
sub_tag = 'university' 
sports_tags = None
gdf_university = get_POIs_from_cities(CITIES,tag_type,sub_tag,sports_tags=None)
display(gdf_university.head())

/root/anaconda3/envs/osmnx/lib/python3.12/site-packages/pandas/core/frame.py:717: DeprecationWarning: Passing a BlockManager to GeoDataFrame is deprecated and will raise in a future version. Use public APIs instead.
  warnings.warn(
/root/anaconda3/envs/osmnx/lib/python3.12/site-packages/pandas/core/frame.py:717: DeprecationWarning: Passing a BlockManager to GeoDataFrame is deprecated and will raise in a future version. Use public APIs instead.
  warnings.warn(


No amenity-university within Décines-Charpieu, France
No amenity-university within Vaulx-En-Velin, France


/root/anaconda3/envs/osmnx/lib/python3.12/site-packages/geopandas/array.py:1459: UserWarning: CRS not set for some of the concatenation inputs. Setting output's CRS as WGS 84 (the single non-null crs provided).
  return GeometryArray(data, crs=_get_common_crs(to_concat))
/root/anaconda3/envs/osmnx/lib/python3.12/site-packages/geopandas/array.py:1459: UserWarning: CRS not set for some of the concatenation inputs. Setting output's CRS as WGS 84 (the single non-null crs provided).
  return GeometryArray(data, crs=_get_common_crs(to_concat))
/root/anaconda3/envs/osmnx/lib/python3.12/site-packages/geopandas/array.py:1459: UserWarning: CRS not set for some of the concatenation inputs. Setting output's CRS as WGS 84 (the single non-null crs provided).
  return GeometryArray(data, crs=_get_common_crs(to_concat))
/root/anaconda3/envs/osmnx/lib/python3.12/site-packages/geopandas/array.py:1459: UserWarning: CRS not set for some of the concatenation inputs. Setting output's CRS as WGS 84 (the sing

,osmid,name,geometry,type,sub_type
0,1614261813,Les studios Lumières,POINT (4.86228 45.74742),amenity,university
1,2392516856,Institut supérieur d'agriculture et d'agroalim...,POINT (4.83626 45.72801),amenity,university
2,3588518600,Université Professionnelle Internationale de Lyon,POINT (4.80590 45.76833),amenity,university
3,3588531207,IDRAC - École de Commerce et de Management,POINT (4.80591 45.76838),amenity,university
4,3588534986,CEFAM,POINT (4.80591 45.76846),amenity,university


### Génère les POIs Nightclub: 

In [15]:
tag_type = 'amenity'
sub_tag = 'nightclub' 
sports_tags = None
gdf_nightclub = get_POIs_from_cities(CITIES,tag_type,sub_tag,sports_tags=None)
display(gdf_nightclub.head())

/root/anaconda3/envs/osmnx/lib/python3.12/site-packages/pandas/core/frame.py:717: DeprecationWarning: Passing a BlockManager to GeoDataFrame is deprecated and will raise in a future version. Use public APIs instead.
  warnings.warn(
/root/anaconda3/envs/osmnx/lib/python3.12/site-packages/pandas/core/frame.py:717: DeprecationWarning: Passing a BlockManager to GeoDataFrame is deprecated and will raise in a future version. Use public APIs instead.
  warnings.warn(


No amenity-nightclub within Décines-Charpieu, France
No amenity-nightclub within Vaulx-En-Velin, France


,osmid,name,geometry,type,sub_type
0,246746973,Fiesta Loca,POINT (4.82653 45.76729),amenity,nightclub
1,248737175,Loft Club,POINT (4.84192 45.74851),amenity,nightclub
2,2062871143,Les Salons du NH,POINT (4.85894 45.73121),amenity,nightclub
3,2283833291,Vertige X,POINT (4.83621 45.76911),amenity,nightclub
4,2543375734,La Cour des Grands,POINT (4.81933 45.75378),amenity,nightclub


## Génère les POI théatre
- 'theatre' -> Bien. Récupère des lieu comme: Transbo, Opera, Thetare romain de fourviere ...

In [16]:
tag_type = 'amenity'
sub_tag = 'theatre' 
sports_tags = None
gdf_theatre = get_POIs_from_cities(CITIES,tag_type,sub_tag,sports_tags=None)
gdf_theatre = gdf_theatre[gdf_theatre.geometry.type != 'Point']
print(gdf_theatre.shape)
display(gdf_theatre.head())

/root/anaconda3/envs/osmnx/lib/python3.12/site-packages/pandas/core/frame.py:717: DeprecationWarning: Passing a BlockManager to GeoDataFrame is deprecated and will raise in a future version. Use public APIs instead.
  warnings.warn(
/root/anaconda3/envs/osmnx/lib/python3.12/site-packages/pandas/core/frame.py:717: DeprecationWarning: Passing a BlockManager to GeoDataFrame is deprecated and will raise in a future version. Use public APIs instead.
  warnings.warn(
/root/anaconda3/envs/osmnx/lib/python3.12/site-packages/pandas/core/frame.py:717: DeprecationWarning: Passing a BlockManager to GeoDataFrame is deprecated and will raise in a future version. Use public APIs instead.
  warnings.warn(


(24, 5)


/root/anaconda3/envs/osmnx/lib/python3.12/site-packages/pandas/core/frame.py:717: DeprecationWarning: Passing a BlockManager to GeoDataFrame is deprecated and will raise in a future version. Use public APIs instead.
  warnings.warn(


,osmid,name,geometry,type,sub_type
42,10061086,Opéra de Lyon,"POLYGON ((4.83691 45.76799, 4.83673 45.76797, ...",amenity,theatre
43,62277297,Théâtre de Guignol,"POLYGON ((4.85371 45.77615, 4.85373 45.77610, ...",amenity,theatre
44,62278804,Bourse du Travail,"POLYGON ((4.84843 45.75943, 4.84825 45.75952, ...",amenity,theatre
45,62336964,Auditorium Maurice-Ravel,"POLYGON ((4.85328 45.76144, 4.85328 45.76145, ...",amenity,theatre
46,62338522,L'assemblée - fabrique artistique,"POLYGON ((4.87326 45.75776, 4.87333 45.75795, ...",amenity,theatre


## Parking 
Notes : `Les Parc Relais sont contenues dans parking`

In [17]:
tag_type = 'amenity'
sub_tag = 'parking'  
sports_tags = None
gdf_parkings = get_POIs_from_cities(CITIES,tag_type,sub_tag,sports_tags=None)
gdf_parkings = gdf_parkings.to_crs('epsg:2154')
gdf_parkings = gdf_parkings[gdf_parkings.geometry.area > 5000]
gdf_parkings = gdf_parkings.to_crs('epsg:4326')
gdf_parkings.head()

/root/anaconda3/envs/osmnx/lib/python3.12/site-packages/pandas/core/frame.py:717: DeprecationWarning: Passing a BlockManager to GeoDataFrame is deprecated and will raise in a future version. Use public APIs instead.
  warnings.warn(
/root/anaconda3/envs/osmnx/lib/python3.12/site-packages/pandas/core/frame.py:717: DeprecationWarning: Passing a BlockManager to GeoDataFrame is deprecated and will raise in a future version. Use public APIs instead.
  warnings.warn(
/root/anaconda3/envs/osmnx/lib/python3.12/site-packages/pandas/core/frame.py:717: DeprecationWarning: Passing a BlockManager to GeoDataFrame is deprecated and will raise in a future version. Use public APIs instead.
  warnings.warn(
/root/anaconda3/envs/osmnx/lib/python3.12/site-packages/pandas/core/frame.py:717: DeprecationWarning: Passing a BlockManager to GeoDataFrame is deprecated and will raise in a future version. Use public APIs instead.
  warnings.warn(
/root/anaconda3/envs/osmnx/lib/python3.12/site-packages/geopandas/ar

,osmid,name,geometry,type,sub_type
83,40649383,NaN,"POLYGON ((4.82936 45.72227, 4.82987 45.72185, ...",amenity,parking
84,44822697,NaN,"POLYGON ((4.82769 45.72638, 4.82871 45.72641, ...",amenity,parking
103,82490523,Bellecour,"POLYGON ((4.83107 45.75857, 4.83103 45.75851, ...",amenity,parking
105,84527723,NaN,"POLYGON ((4.82753 45.72885, 4.82755 45.72925, ...",amenity,parking
111,88806907,Parc Relais TCL Parilly,"POLYGON ((4.88538 45.72042, 4.88626 45.72099, ...",amenity,parking


### Train Station in Lyon

In [18]:
tag_type = 'building'
sub_tag = 'train_station'  
sports_tags = None
gdf_train = get_POIs_from_cities(CITIES,tag_type,sub_tag,sports_tags=None)
display(gdf_train)

No building-train_station within Villeurbanne, France
No building-train_station within Décines-Charpieu, France
No building-train_station within Vaulx-En-Velin, France


/root/anaconda3/envs/osmnx/lib/python3.12/site-packages/pandas/core/frame.py:717: DeprecationWarning: Passing a BlockManager to GeoDataFrame is deprecated and will raise in a future version. Use public APIs instead.
  warnings.warn(


,osmid,name,geometry,type,sub_type
0,31363448,NaN,"POLYGON ((4.80444 45.78067, 4.80460 45.78084, ...",building,train_station
1,35143415,Gare de Lyon Perrache,"POLYGON ((4.82563 45.74864, 4.82565 45.74865, ...",building,train_station
2,62337529,NaN,"POLYGON ((4.86151 45.76039, 4.86147 45.76039, ...",building,train_station
0,196590753,Hall 1,"POLYGON ((4.86125 45.76035, 4.86124 45.76035, ...",building,train_station


## IRIS Zone

In [19]:
PATH_iris = f'{FOLDER_PATH}/lyon_iris_shapefile'
gdf_iris = gpd.read_file(f'{PATH_iris}/lyon.shp')
gdf_iris = gdf_iris[['CODE_IRIS','NOM_IRIS','geometry']].rename(columns = {'CODE_IRIS':'osmid','NOM_IRIS':'name'})
gdf_iris['type'] = 'IRIS'
gdf_iris['sub_type'] = 'IRIS'
display(gdf_iris.head())

,osmid,name,geometry,type,sub_type
0,692860202.0,Mont Blanc,"POLYGON ((4.89204 45.81283, 4.89271 45.81305, ...",IRIS,IRIS
1,693820501.0,Montrochet-Marché-Gare,"POLYGON ((4.82108 45.75066, 4.82109 45.75066, ...",IRIS,IRIS
2,692750107.0,Bonneveau,"POLYGON ((4.96062 45.76006, 4.96035 45.76008, ...",IRIS,IRIS
3,693860201.0,Puvis de Chavannes,"POLYGON ((4.84567 45.77084, 4.84465 45.77073, ...",IRIS,IRIS
4,691490303.0,Montmein,"POLYGON ((4.80838 45.71222, 4.80848 45.71214, ...",IRIS,IRIS


# Autres: 
- 'car_pooling' -> 1 seul quais perrache 
- 'waiting_area' -> 1 seule gare perrache 
- 'music_venue' -> 2 endroit seulement vers croix paquet (Collision et Kraspek ? )
- 'marketplace' -> Tout les marché alimentaire  + Puces, pas mal aussi 
- 'cinema'   -> Beaucoup de ciné, pas sur que ce soit utile 
- 'parking'  -> Contient également les park relais. Aucun moyen de les identifier, donc je ne considère seulement des parking qui soient suffisamment gros.
- 'social_facility -> Contient beaucoup d'Ephad, pas sur que ce soit utile non plus. 
- Etonnament aucun moyen d'identifier les salles de concerts.

In [12]:
all_tag.amenity.unique()  

array(['fuel', 'marketplace', 'post_office', 'restaurant', 'post_box',
       'cinema', 'pub', 'bicycle_rental', 'parking', 'school', 'library',
       'pharmacy', 'fast_food', 'cafe', 'bank', 'bar', 'toilets',
       'bicycle_parking', 'parking_entrance', 'police', 'nightclub',
       'social_centre', 'drinking_water', 'social_facility', 'recycling',
       'theatre', 'motorcycle_parking', 'bench', 'hospital',
       'fire_station', 'atm', 'grit_bin', 'waste_basket', 'clock',
       'vending_machine', 'fountain', 'parking_space', 'kindergarten',
       'taxi', 'college', 'restaurant;club', 'vehicle_inspection',
       'community_centre', 'waste_disposal', 'driving_school', 'car_wash',
       'doctors', 'university', 'place_of_worship', 'car_sharing',
       'hookah_lounge', 'veterinary', 'music_venue', 'photo_booth',
       'clinic', 'stripclub', 'money_transfer', 'animal_boarding',
       'dentist', 'car_rental', 'ice_cream', 'biergarten', 'childcare',
       'swingerclub', 'arts_cen

### Save and Load : 

In [20]:
# Save : 
for name in ['park','stadium','university','shop','nightclub','parkings','train','theatre','iris']:
    save_path = f"{FOLDER_PATH}/POIs"
    print('save path : ',save_path)
    if not os.path.exists(save_path):
        os.mkdir(save_path)
    globals()[f"gdf_{name}"].to_file(f"{save_path}/gdf_{name}.geojson", driver='GeoJSON')


# Load : 
if False:
    save_path = f"{FOLDER_PATH}/POIs"
    gpd.read_file(f"{save_path}/gdf_stadium.geojson")

save path :  ../../../../data/rrochas/prediction_validation/POIs
save path :  ../../../../data/rrochas/prediction_validation/POIs
save path :  ../../../../data/rrochas/prediction_validation/POIs
save path :  ../../../../data/rrochas/prediction_validation/POIs
save path :  ../../../../data/rrochas/prediction_validation/POIs
save path :  ../../../../data/rrochas/prediction_validation/POIs
save path :  ../../../../data/rrochas/prediction_validation/POIs
save path :  ../../../../data/rrochas/prediction_validation/POIs
save path :  ../../../../data/rrochas/prediction_validation/POIs


# Limites Observées : 
- Il manques des POIs, en particulier car je travaille avec des données 2019 et les POIs sont ceux actuels. Certains bars/boites ont ouvert depuis, d'autres ont fermé (cas du Ninkasi Kao lorsqu'on repère des anomalies de prédiction autour de la station Gerland). Les aménagements ont donc largement évolué, ce qui induis des biais/erreurs.

In [48]:
gdf_filtered = gdf[#(gdf.amenity =='pub') |
    #(gdf.amenity =='bar') |
    (gdf.amenity =='nightclub') 
    #(gdf.amenity =='restaurant') |
    #(gdf.amenity =='music_venue')
    ][['amenity','name','geometry']]

# Add NinKasi Kao which doesn't existe here (closed since)
from shapely import Point 
ninkasi_kao = [4.830913047430858,45.72818916789741]
geom = Point(ninkasi_kao)

gdf_nighclub = gpd.GeoDataFrame(pd.concat([gdf_filtered, pd.DataFrame({'amenity': ['nightclub'],'name':['Ninkasi Kao'], 'geometry':[geom]})],ignore_index=True))

# Save:
if False :
    FOLDER_PATH = '../../../../data/rrochas/prediction_validation' 
    save_path = f"{FOLDER_PATH}/POIs"
    gdf_nighclub.to_file(f"{save_path}/gdf_nightclub.geojson", driver='GeoJSON')

# gdf_filtered.explore('amenity', marker_kwds=dict(radius=7)) 

/root/anaconda3/envs/osmnx/lib/python3.12/site-packages/pandas/core/frame.py:717: DeprecationWarning: Passing a BlockManager to GeoDataFrame is deprecated and will raise in a future version. Use public APIs instead.
  warnings.warn(
/root/anaconda3/envs/osmnx/lib/python3.12/site-packages/pandas/core/frame.py:717: DeprecationWarning: Passing a BlockManager to GeoDataFrame is deprecated and will raise in a future version. Use public APIs instead.
  warnings.warn(
/root/anaconda3/envs/osmnx/lib/python3.12/site-packages/pandas/core/frame.py:717: DeprecationWarning: Passing a BlockManager to GeoDataFrame is deprecated and will raise in a future version. Use public APIs instead.
  warnings.warn(
/root/anaconda3/envs/osmnx/lib/python3.12/site-packages/pandas/core/frame.py:717: DeprecationWarning: Passing a BlockManager to GeoDataFrame is deprecated and will raise in a future version. Use public APIs instead.
  warnings.warn(
/root/anaconda3/envs/osmnx/lib/python3.12/site-packages/pandas/core/

In [16]:

"""
osmid: 
Gare Part-dieu :  [196590753,62337529,1039333290,1191457432,1158815494]
Gare Perrache: [35143415]
Gare Vaise [31363448]
"""

#part_dieu_ids = [196590753,62337529,1039333290,1191457432,1158815494]
#gdf_train_station = gdf_train_station.reset_index()
#gdf_train_station[gdf_train_station['osmid'].isin([part_dieu_ids])].explore()



'\nosmid: \nGare Part-dieu :  [196590753,62337529,1039333290,1191457432,1158815494]\nGare Perrache: [35143415]\nGare Vaise [31363448]\n'

In [11]:
place_name = f"Lyon, France"
gdf_subway_station = ox.geometries_from_place(place_name, tags ={"railway": "station"})
gdf_subway_station.head()

/tmp/ipykernel_85790/305611499.py:2: FutureWarning: The `geometries` module and `geometries_from_X` functions have been renamed the `features` module and `features_from_X` functions. Use these instead. The `geometries` module and function names are deprecated and will be removed in the v2.0.0 release. See the OSMnx v2 migration guide: https://github.com/gboeing/osmnx/issues/1123
  gdf_subway_station = ox.geometries_from_place(place_name, tags ={"railway": "station"})
/root/anaconda3/envs/osmnx/lib/python3.12/site-packages/pandas/core/frame.py:717: DeprecationWarning: Passing a BlockManager to GeoDataFrame is deprecated and will raise in a future version. Use public APIs instead.
  warnings.warn(
/root/anaconda3/envs/osmnx/lib/python3.12/site-packages/pandas/core/frame.py:717: DeprecationWarning: Passing a BlockManager to GeoDataFrame is deprecated and will raise in a future version. Use public APIs instead.
  warnings.warn(
/root/anaconda3/envs/osmnx/lib/python3.12/site-packages/pandas

source                  geometry level  \
element_type osmid                                                        
node         4290857016             NaN  POINT (4.86016 45.76057)   NaN   
             4290857018  SNCF - 06/2013  POINT (4.82533 45.74832)   NaN   
             5365148732             NaN  POINT (4.83301 45.75736)   NaN   
             5365152101             NaN  POINT (4.82596 45.76017)   NaN   
             5365157959             NaN  POINT (4.83602 45.76786)   NaN   

                         railway wheelchair iata importance  \
element_type osmid                                            
node         4290857016  station        yes  XYD   national   
             4290857018  station        yes  NaN        NaN   
             5365148732  station        yes  NaN        NaN   
             5365152101  station        NaN  NaN        NaN   
             5365157959  station        yes  NaN        NaN   

                                                       name  \
element_type osmid                                            
node         4290857016                      Lyon Part-Dieu   
             4290857018                       Lyon-Perrache   
             5365148732                           Bellecour   
             5365152101  Vieux Lyon - Cathédrale Saint-Jean   
             5365157959       Hôtel de Ville - Louis Pradel   

                                                                      note  \
element_type osmid                                                           
node         4290857016  Please do not add the tag public_transport=sta...   
             4290857018  Please do not add the tag public_transport=sta...   
             5365148732                                                NaN   
             5365152101                                                NaN   
             5365157959                                                NaN   

                        operator  ... official_name tactile_paving area  \
element_type osmid                ...                                     
node         4290857016     SNCF  ...           NaN            NaN  NaN   
             4290857018     SNCF  ...           NaN            NaN  NaN   
             5365148732      NaN  ...           NaN            NaN  NaN   
             5365152101      NaN  ...           NaN            NaN  NaN   
             5365157959      NaN  ...           NaN            NaN  NaN   

                        alt_name ref:FR:FANTOIR layer location max_level  \
element_type osmid                                                         
node         4290857016      NaN            NaN   NaN      NaN       NaN   
             4290857018      NaN            NaN   NaN      NaN       NaN   
             5365148732      NaN            NaN   NaN      NaN       NaN   
             5365152101      NaN            NaN   NaN      NaN       NaN   
             5365157959      NaN            NaN   NaN      NaN       NaN   

                        min_level building:levels:underground  
element_type osmid                                             
node         4290857016       NaN                         NaN  
             4290857018       NaN                         NaN  
             5365148732       NaN                         NaN  
             5365152101       NaN                         NaN  
             5365157959       NaN                         NaN  

[5 rows x 40 columns]

In [12]:
place_name = f"Lyon, France"
ox.geometries_from_place(place_name, tags ={"railway": "subway"}).explore()

/tmp/ipykernel_85790/1272158680.py:2: FutureWarning: The `geometries` module and `geometries_from_X` functions have been renamed the `features` module and `features_from_X` functions. Use these instead. The `geometries` module and function names are deprecated and will be removed in the v2.0.0 release. See the OSMnx v2 migration guide: https://github.com/gboeing/osmnx/issues/1123
  ox.geometries_from_place(place_name, tags ={"railway": "subway"}).explore()
/root/anaconda3/envs/osmnx/lib/python3.12/site-packages/pandas/core/frame.py:717: DeprecationWarning: Passing a BlockManager to GeoDataFrame is deprecated and will raise in a future version. Use public APIs instead.
  warnings.warn(
/root/anaconda3/envs/osmnx/lib/python3.12/site-packages/pandas/core/frame.py:717: DeprecationWarning: Passing a BlockManager to GeoDataFrame is deprecated and will raise in a future version. Use public APIs instead.
  warnings.warn(
/root/anaconda3/envs/osmnx/lib/python3.12/site-packages/pandas/core/frame